# Реализация DEFLATE

## Импорт текста и декапитализированного варианта

In [60]:
filename = 'text.txt'
with open(filename, 'r', encoding='ASCII') as f:
    text = f.read()

In [61]:
with open('decapitalized.txt', 'r', encoding='ASCII') as f:
    decapitalized_text = f.read()

### Статическое дерево Хаффмана

In [62]:
from heapq import heappush, heappop
from collections import Counter


def huffman(frequencies: dict) -> list:
    """Создание статического дерева Хаффмана на основе словаря частот

    Пример логики работы:  
    frequencies = {'a': 10, 'b': 3, 'c': 2}  
    queue = [(2, 'c'), (3, 'b'), (10, 'a')]  
    res = {'a': '', 'b': '', 'c': ''}  

    1 шаг:  
    first_freq, first_letters = 2, 'c',  
    second_freq, second_letters = 3, 'b'  
      
    res = {'a': '', 'b': '1', 'c': '0'}  
    queue = [(5, 'cb'), (10, 'a')]  
  
    2 шаг:  
    first_freq, first_letters = 5, 'cb',  
    second_freq, second_letters = 10, 'a'  
      
    res = {'a': '1', 'b': '01', 'c': '00'}  
    queue = [(15, 'cba')]  
  
    Общий смысл - сопоставить наиболее редким символам наиболее длинный код  
    и, наоборот, наиболее частым - наиболее короткий  
    """

    # Для текста с алфавитом из одного символа кодируем его нулём
    if len(frequencies) == 1:
        letter, = frequencies
        return {letter: '0'}

    # инициализируем очередь, которая будет содержать в начале символы с минимальной частотой (мин-куча)
    queue = []
    res = {letter: '' for letter in frequencies} # инициализируем код для каждой частоты

    for letter, frequency in frequencies.items():
        heappush(queue, (frequency, letter)) # делаем мин-кучу из пустого списка

    while len(queue) > 1:
        first_freq, first_letters = heappop(queue) # достаём самый редкий символ из кучи и его частоту, удаляя из кучи
        second_freq, second_letters = heappop(queue) # достаём второй самый редкий символ из кучи и его частоту, удаляя из кучи

        for letter in first_letters:
            res[letter] = '0' + res[letter] # для самого редкого элемента прибавляем ноль в начало кода для каждого символа, из которого он состоит

        # Пример: для first_letters = 'abc' прибавится 0 в начало кода для a, b, c.

        for letter in second_letters:
            res[letter] = '1' + res[letter] # для второго самого редкого элемента прибавляем единицу в начале кода для каждого символа, из которого он состоит


        # Закидываем в мин-кучу объединённый элемент из прошлых двух: сумма их частот и сумма их названий ('a': 3, 'b':4 => (7, 'ab'))
        heappush(
            queue,
            (
                first_freq + second_freq,
                ''.join(sorted(first_letters + second_letters))
            )
        )

    return res

def count_tree_lenght(tree: dict) -> int:
    # Кодируем каждый символ одним байтом, 
    return len(tree.keys()) * 8 + len(tree.values()) * 32

In [63]:
frequencies = {'a': 10, 'b': 3, 'c': 2, 'd': 32, 'f': 13, 'k': 44}
huffman(frequencies)

{'a': '1011', 'b': '10101', 'c': '10100', 'd': '11', 'f': '100', 'k': '0'}

## Кодировщики 

### Енкодер букв и длин в единый алфавит, который будет использоваться для построения первого дерева Хаффмана

In [64]:
length_codes = [
    (3, 0),     # 257
    (4, 0),     # 258
    (5, 0),     # 259
    (6, 0),     # 260
    (7, 0),     # 261
    (8, 0),     # 262
    (9, 0),     # 263
    (10, 0),    # 264
    (11, 1),    # 265 (11-12)
    (13, 1),    # 266 (13-14)
    (15, 1),    # 267 (15-16)
    (17, 1),    # 268 (17-18)
    (19, 2),    # 269 (19-22)
    (23, 2),    # 270 (23-26)
    (27, 2),    # 271 (27-30)
    (31, 2),    # 272 (31-34)
    (35, 3),    # 273 (35-42)
    (43, 3),    # 274 (43-50)
    (51, 3),    # 275 (51-58)
    (59, 3),    # 276 (59-66)
    (67, 4),    # 277 (67-82)
    (83, 4),    # 278 (83-98)
    (99, 4),    # 279 (99-114)
    (115, 4),   # 280 (115-130)
    (131, 5),   # 281 (131-162)
    (163, 5),   # 282 (163-194)
    (195, 5),   # 283 (195-226)
    (227, 5),   # 284 (227-257)
    (258, 0)    # 285
]

In [ ]:
def encode_symbol(symbol, length_codes):
    """Кодирует символ (букву или длину) в соответствующий код."""
    if isinstance(symbol, str):
        return ord(symbol)
    
    elif isinstance(symbol, int):
        if not 3 <= symbol <= 258:
            raise ValueError("Length must be in 3-258 range")
        
        for i in range(len(length_codes)):
            # Проверяем верхнюю границу (если не последний элемент)
            if i < len(length_codes) - 1 and symbol >= length_codes[i+1][0]:
                continue
            
            difference = symbol - length_codes[i][0]
            return (257 + i, bin(difference)[2:])
    
    else:
        raise TypeError("Symbol must be str (letter) or int (length)")

In [66]:
print(encode_symbol(258, length_codes))

(285, '0')


In [67]:
print(ord('\n'))
print(encode_symbol('\n', length_codes))

10
10


### Енкодер смещений, который будет использоваться для построения второго дерева Хаффмана

In [68]:
distance_codes = [
    (1, 0),       # код 0
    (2, 0),       # код 1
    (3, 0),       # код 2
    (4, 0),       # код 3
    (5, 1),       # код 4 (5-6)
    (7, 1),       # код 5 (7-8)
    (9, 2),       # код 6 (9-12)
    (13, 2),      # код 7 (13-16)
    (17, 3),      # код 8 (17-24)
    (25, 3),      # код 9 (25-32)
    (33, 4),      # код 10 (33-48)
    (49, 4),      # код 11 (49-64)
    (65, 5),      # код 12 (65-96)
    (97, 5),      # код 13 (97-128)
    (129, 6),     # код 14 (129-192)
    (193, 6),     # код 15 (193-256)
    (257, 7),     # код 16 (257-384)
    (385, 7),     # код 17 (385-512)
    (513, 8),     # код 18 (513-768)
    (769, 8),     # код 19 (769-1024)
    (1025, 9),    # код 20 (1025-1536)
    (1537, 9),    # код 21 (1537-2048)
    (2049, 10),   # код 22 (2049-3072)
    (3073, 10),   # код 23 (3073-4096)
    (4097, 11),   # код 24 (4097-6144)
    (6145, 11),   # код 25 (6145-8192)
    (8193, 12),   # код 26 (8193-12288)
    (12289, 12),  # код 27 (12289-16384)
    (16385, 13),  # код 28 (16385-24576)
    (24577, 13),  # код 29 (24577-32768)
    (32769, 14),  # код 30 (32769-49152) (Расширили для случая 64кб длины окна)
    (49153, 14),  # код 31 (49153-65536) (Расширили для случая 64кб длины окна)
]

In [69]:
def encode_distance(distance: int, distance_codes: list) -> tuple:
    for i in range(len(distance_codes)):
        # Проверяем верхнюю границу (если не последний элемент)
        if i < len(distance_codes) - 1 and distance >= distance_codes[i+1][0]:
            continue
        
        difference = distance - distance_codes[i][0]
        return (i, bin(difference)[2:].zfill(distance_codes[i][1]))
    
    raise ValueError(f"Invalid distance: {distance}")

In [70]:
encode_distance(49154, distance_codes)

(31, '00000000000001')

## LZ77

### Version 1

In [123]:
class Lz77Compressor:
    def __init__(self, window_size: int, data, min_match_len=3, max_match_len=258):
        if window_size > 0 and window_size <= 64 * 1024:
            self.window_size = window_size
        else:
            raise ValueError('Неверно указана ширина окна. Возможные значения: от 1 до 64кб')
        
        self.data = data
        self.min_match_len = min_match_len
        self.max_match_len = max_match_len
        self.hash_table = {}  # ключ: 3-символьная строка, значение: список позиций

    def compress(self):
        pos = 0
        output = []
        
        while pos < len(self.data):
            # Определяем границы текущего буфера
            buffer_start = max(0, pos - self.window_size)
            
            # Получаем текущий префикс (3 символа)
            if pos + 3 > len(self.data):
                output.append(self.data[pos])
                pos += 1
                continue
                
            prefix = self.data[pos:pos+3]
            
            # Ищем совпадения в хэш-таблице
            match_offset, match_len = self._find_longest_match(pos, buffer_start)
            
            if match_len >= self.min_match_len:
                output.append((match_offset, match_len))
                # Добавляем в хэш-таблицу все новые 3-символьные подстроки из найденной фразы
                for i in range(pos, pos + match_len):
                    if i + 3 <= len(self.data):
                        new_prefix = self.data[i:i+3]
                        self.hash_table.setdefault(new_prefix, []).append(i)
                pos += match_len
            else:
                output.append(self.data[pos])
                # Добавляем текущий префикс в хэш-таблицу
                self.hash_table.setdefault(prefix, []).append(pos)
                pos += 1
                
        return output

    def _find_longest_match(self, pos, buffer_start):
        max_len = 0
        max_offset = 0
        
        if pos + 3 > len(self.data):
            return 0, 0
            
        prefix = self.data[pos:pos+3]
        candidates = self.hash_table.get(prefix, [])
        
        for candidate_pos in reversed(candidates):
            # Пропускаем кандидатов вне текущего буфера
            if candidate_pos < buffer_start:
                break
                
            offset = pos - candidate_pos
            length = 3  # уже знаем, что первые 3 символа совпадают
            
            # Сравниваем остальные символы
            max_possible = min(self.max_match_len, len(self.data) - pos)
            while (length < max_possible and 
                   self.data[candidate_pos + length] == self.data[pos + length]):
                length += 1
                
            if length > max_len:
                max_len = length
                max_offset = offset
                
        return max_offset, max_len

In [124]:
data = text
compressor = Lz77Compressor(window_size=32 * 1024, data=data)
result = compressor.compress()
print(result[:100])

[' ', (1, 37), '1', '7', '8', '1', '\n', '\n', (32, 26), 'T', 'H', 'E', ' ', 'C', 'R', 'I', 'T', 'I', 'Q', 'U', 'E', ' ', 'O', 'F', ' ', 'P', 'U', 'R', 'E', ' ', 'R', 'E', 'A', 'S', 'O', 'N', (55, 28), (1, 6), 'b', 'y', ' ', 'I', 'm', 'm', 'a', 'n', 'u', 'e', 'l', ' ', 'K', 'a', 'n', 't', (50, 25), 't', 'r', 'a', 'n', 's', 'l', 'a', 't', 'e', 'd', (52, 4), 'J', '.', ' ', 'M', '.', ' ', 'D', (6, 3), 'e', 'i', 'k', 'l', 'e', 'j', 'o', 'h', 'n', (58, 14), 'P', 'R', 'E', 'F', 'A', 'C', 'E', ' ', 'T', 'O', (160, 5), 'F', 'I', 'R', 'S', 'T']


### Version 2 (+ Трюк с улучшением поиска)

In [ ]:
class Lz77Compressor_Tricked:
    def __init__(self, window_size: int, data, min_match_len=3, max_match_len=258):
        if 0 < window_size <= 64 * 1024:
            self.window_size = window_size
        else:
            raise ValueError('Неверно указана ширина окна. Возможные значения: от 1 до 64КБ')
        
        self.data = data
        self.min_match_len = min_match_len
        self.max_match_len = max_match_len
        self.hash_table = {}  # ключ: 3-символьная строка, значение: список позиций

    def compress(self):
        pos = 0
        output = []
        
        while pos < len(self.data):
            buffer_start = max(0, pos - self.window_size)
            
            if pos + self.min_match_len > len(self.data):
                output.append(self.data[pos])
                pos += 1
                continue

            prefix = self.data[pos:pos+self.min_match_len]
            match_offset, match_len = self._find_longest_match(pos, buffer_start)

            # Если найден повтор, делаем повтор снова, начиная со следующей позиции
            if match_len >= self.min_match_len:
                lookahead_better = False
                if pos + 1 + self.min_match_len <= len(self.data):
                    next_prefix = self.data[pos+1: pos+1+self.min_match_len]
                    if next_prefix in self.hash_table:
                        next_offset, next_len = self._find_longest_match(pos + 1, buffer_start)
                        # Если найден более длинный повтор, начиная со след. позиции, записываем литерал и более длинный повтор
                        if next_len > match_len: # Здесь можно добавить порог (next_len > match_len + 5)
                            output.append(self.data[pos])  # Записываем одинокий литерал
                            self._add_prefix(pos)
                            pos += 1
                            output.append((pos - next_offset, next_len)) # Записываем новый более длинный повтор
                            self._add_match_to_table(pos, next_len)
                            pos += next_len
                            lookahead_better = True
                
                # Если трюк не дал результат лучше, добавляем имеющиеся данные
                if not lookahead_better:
                    output.append((match_offset, match_len))
                    self._add_match_to_table(pos, match_len)
                    pos += match_len
            else:
                output.append(self.data[pos])
                self._add_prefix(pos)
                pos += 1

        return output

    def _find_longest_match(self, pos, buffer_start):
        max_len = 0
        max_offset = 0

        if pos + self.min_match_len > len(self.data):
            return 0, 0

        prefix = self.data[pos:pos+self.min_match_len]
        candidates = self.hash_table.get(prefix, [])

        for candidate_pos in reversed(candidates):
            if candidate_pos < buffer_start:
                break

            offset = pos - candidate_pos
            length = self.min_match_len
            max_possible = min(self.max_match_len, len(self.data) - pos)

            while (length < max_possible and
                   self.data[pos + length] == self.data[candidate_pos + length]):
                length += 1

            if length > max_len:
                max_len = length
                max_offset = offset

        return max_offset, max_len

    def _add_prefix(self, pos):
        """Добавить текущий 3-символьный префикс в хэш-таблицу"""
        if pos + self.min_match_len <= len(self.data):
            prefix = self.data[pos:pos+self.min_match_len]
            self.hash_table.setdefault(prefix, []).append(pos)

    def _add_match_to_table(self, pos, match_len):
        """Добавить все 3-символьные подстроки фразы в таблицу"""
        for i in range(pos, pos + match_len):
            if i + self.min_match_len <= len(self.data):
                prefix = self.data[i:i+self.min_match_len]
                self.hash_table.setdefault(prefix, []).append(i)


In [192]:
data = text
compressor = Lz77Compressor_Tricked(window_size=64 * 1024, data=data)
result = compressor.compress()
print(result[:100])

[' ', (1, 37), '1', '7', '8', '1', '\n', '\n', (32, 26), 'T', 'H', 'E', ' ', 'C', 'R', 'I', 'T', 'I', 'Q', 'U', 'E', ' ', 'O', 'F', ' ', 'P', 'U', 'R', 'E', ' ', 'R', 'E', 'A', 'S', 'O', 'N', (55, 28), (1, 6), 'b', 'y', ' ', 'I', 'm', 'm', 'a', 'n', 'u', 'e', 'l', ' ', 'K', 'a', 'n', 't', (50, 25), 't', 'r', 'a', 'n', 's', 'l', 'a', 't', 'e', 'd', (52, 4), 'J', '.', ' ', 'M', '.', ' ', 'D', (6, 3), 'e', 'i', 'k', 'l', 'e', 'j', 'o', 'h', 'n', (58, 14), 'P', 'R', 'E', 'F', 'A', 'C', 'E', ' ', 'T', 'O', (160, 5), 'F', 'I', 'R', 'S', 'T']


## Разбиение результата на последовательности символов/длин и смещений

In [193]:
literals_or_lengths = []
distances = []

for entry in result:
    if isinstance(entry, tuple):
        offset, length = entry
        literals_or_lengths.append(length)
        distances.append(offset)
    else:
        literals_or_lengths.append(entry)

In [194]:
print(f'Список литералов или длин: {literals_or_lengths[:10]}, его длина = {len(literals_or_lengths)}')
print(f'Список расстояний: {distances[:10]}, его длина = {len(distances)}')

Список литералов или длин: [' ', 37, '1', '7', '8', '1', '\n', '\n', 26, 'T'], его длина = 168938
Список расстояний: [1, 32, 55, 1, 50, 52, 6, 58, 160, 166], его длина = 136680


## Кодируем полученные последовательности и считаем частоты кодов для передачи в статические деревья Хаффмана

In [195]:
symbol_frequencies = Counter()
distance_frequencies = Counter()

for entry in result:  # result — исходный список LZ77 (литералы/кортежи)
    if isinstance(entry, tuple):
        offset, length = entry
        
        # Кодируем длину и добавляем в symbol_frequencies
        code, _ = encode_symbol(length, length_codes)
        symbol_frequencies[code] += 1

        # Кодируем расстояние и добавляем в distance_frequencies
        dist_code, _ = encode_distance(offset, distance_codes)
        distance_frequencies[dist_code] += 1
            
    else:
        # Кодируем литерал
        code = encode_symbol(entry, length_codes)
        symbol_frequencies[code] += 1

In [196]:
print(symbol_frequencies.keys())
len(symbol_frequencies.values())

dict_keys([32, 273, 49, 55, 56, 10, 270, 84, 72, 69, 67, 82, 73, 81, 85, 79, 70, 80, 65, 83, 78, 271, 260, 98, 121, 109, 97, 110, 117, 101, 108, 75, 116, 114, 115, 100, 258, 74, 46, 77, 68, 257, 105, 107, 106, 111, 104, 266, 259, 44, 263, 112, 102, 99, 103, 113, 119, 268, 264, 118, 262, 261, 120, 87, 265, 66, 59, 45, 58, 269, 42, 91, 34, 93, 122, 76, 267, 40, 41, 39, 63, 53, 50, 71, 272, 86, 61, 88, 43, 62, 51, 89, 52, 274, 54, 57, 48, 275, 278, 276, 90, 125, 33])


103

## Деревья Хаффмана

Перепишем логику построения деревьев Хаффмана для случая подачи кодов на вход

In [197]:
from heapq import heappush, heappop

def huffman(frequencies: dict) -> dict:
    """Создание дерева Хаффмана для числовых символов"""

    if len(frequencies) == 1:
        only_symbol = next(iter(frequencies))
        return {only_symbol: '0'}

    queue = []
    res = {symbol: '' for symbol in frequencies}

    for symbol, freq in frequencies.items():
        heappush(queue, (freq, [symbol]))  # список из одного символа (тут изменение symbol -> [symbol])

    while len(queue) > 1:
        freq1, symbols1 = heappop(queue) # например: 50, 32
        freq2, symbols2 = heappop(queue) # например: 40, 64

        for sym in symbols1:
            res[sym] = '0' + res[sym]
        for sym in symbols2:
            res[sym] = '1' + res[sym]

        merged_symbols = symbols1 + symbols2 # Получается список из объединённых кодов символов, например: [32, 64]. Затем уже для каждого кода из списка будет добавляться 0 или 1
        heappush(queue, (freq1 + freq2, merged_symbols)) #(тут изменение. Пушим не конкатенацию строк, а объединённый список)

    return res


In [198]:
symbol_huffman_codes = huffman(symbol_frequencies)
distance_huffman_codes = huffman(distance_frequencies)

In [199]:
from collections import Counter

def compress_data(literals_or_lengths, distances, 
                 length_codes, distance_codes,
                 encode_symbol, encode_distance,
                 symbol_huffman_codes, distance_huffman_codes):
    """
    Финальный этап компрессии с битовой записью
    
    Параметры:
        literals_or_lengths: список литералов (str) и длин (int)
        distances: список расстояний (только для кортежей)
        length_codes: таблица кодирования длин
        distance_codes: таблица кодирования расстояний
        encode_symbol: функция кодирования символа/длины
        encode_distance: функция кодирования расстояния
        symbol_huffman_codes: готовые коды Хаффмана для символов/длин
        distance_huffman_codes: готовые коды Хаффмана для расстояний
    """
    
    class BitWriter:
        def __init__(self):
            self.buffer = 0      # Бит-буфер
            self.bit_count = 0   # Текущее количество бит в буфере
            self.output = bytearray()

        def write_bits(self, value, num_bits):
            """Запись битов в буфер"""
            self.buffer = (self.buffer << num_bits) | value
            self.bit_count += num_bits
            while self.bit_count >= 8:
                self.output.append((self.buffer >> (self.bit_count - 8)) & 0xFF)
                self.bit_count -= 8
                self.buffer &= (1 << self.bit_count) - 1

        def flush(self):
            """Завершаем запись (дополняем нулями)"""
            if self.bit_count > 0:
                self.output.append((self.buffer << (8 - self.bit_count)) & 0xFF)
            return bytes(self.output)

    writer = BitWriter()
    dist_index = 0  # Индекс в массиве расстояний, чтобы записывать код расстояния сразу после записи кода длины

    for item in literals_or_lengths:
        if isinstance(item, int): # Встретили число => кодируем длину и расстояние (смещение)
            # Кодируем длину
            code, extra_bits = encode_symbol(item, length_codes)
            # Получаем код из дерева Хаффмана для полученного кода длины и записываем его
            huff_code = symbol_huffman_codes[code]
            writer.write_bits(int(huff_code, 2), len(huff_code))
            
            # Записываем дополнительные биты длины, если есть
            if extra_bits:
                writer.write_bits(int(extra_bits, 2), len(extra_bits))

            # Кодируем расстояние
            dist = distances[dist_index]
            dist_code, dist_extra = encode_distance(dist, distance_codes)
            # Получаем код из дерева Хаффмана для полученного кода смещения и записываем его
            dist_huff = distance_huffman_codes[dist_code]
            writer.write_bits(int(dist_huff, 2), len(dist_huff))
            
            # Записываем дополнительные биты расстояния
            if dist_extra:
                writer.write_bits(int(dist_extra, 2), len(dist_extra))
            
            dist_index += 1

        else:  # Литерал
            code = encode_symbol(item, length_codes)
            huff_code = symbol_huffman_codes[code]
            writer.write_bits(int(huff_code, 2), len(huff_code))

    return writer.flush()

In [200]:
compressed_data = compress_data(
    literals_or_lengths=literals_or_lengths,
    distances=distances,
    length_codes=length_codes,
    distance_codes=distance_codes,
    encode_symbol=encode_symbol,
    encode_distance=encode_distance,
    symbol_huffman_codes=symbol_huffman_codes,
    distance_huffman_codes=distance_huffman_codes
)

In [201]:
print(f'Размер сжатого текста = {len(compressed_data) / 1024:.2f} кб')

Размер сжатого текста = 388.35 кб


## Сбор всего вместе и анализ влияния параметров на скорость и степень сжатия

In [118]:
def deflate_compressor(data, compressor='usual', window_size=32*1024):
    if compressor == 'usual':
        compressor = Lz77Compressor(window_size=window_size, data=data)
    elif compressor == 'tricked':
        compressor = Lz77Compressor_Tricked(window_size=window_size, data=data)
    else:
        raise ValueError('Выберите компрессор: usual или tricked')
    
    result = compressor.compress()  # список литералов и кортежей (offset, length)

    # 2. Подготавливаем списки для дальнейшего кодирования
    literals_or_lengths = []
    distances = []

    for entry in result:
        if isinstance(entry, tuple):
            offset, length = entry
            literals_or_lengths.append(length)  # длина фразы
            distances.append(offset)            # смещение
        else:
            literals_or_lengths.append(entry)   # литерал (символ)

    # 3. Подсчитываем частоты кодов для Хаффмана
    symbol_frequencies = Counter()
    distance_frequencies = Counter()

    for entry in result:
        if isinstance(entry, tuple):
            offset, length = entry

            # Кодируем длину и увеличиваем частоту символа
            code, _ = encode_symbol(length, length_codes)
            symbol_frequencies[code] += 1

            # Кодируем расстояние и увеличиваем частоту
            dist_code, _ = encode_distance(offset, distance_codes)
            distance_frequencies[dist_code] += 1
        else:
            # Литерал — кодируем напрямую
            code = encode_symbol(entry, length_codes)
            symbol_frequencies[code] += 1

    # 4. Строим таблицы кодов Хаффмана для длины и расстояния
    symbol_huffman_codes = huffman(symbol_frequencies)
    distance_huffman_codes = huffman(distance_frequencies)

    # 5. Генерируем окончательный сжатый поток
    compressed_data = compress_data(
        literals_or_lengths=literals_or_lengths,
        distances=distances,
        length_codes=length_codes,
        distance_codes=distance_codes,
        encode_symbol=encode_symbol,
        encode_distance=encode_distance,
        symbol_huffman_codes=symbol_huffman_codes,
        distance_huffman_codes=distance_huffman_codes
    )

    return compressed_data

In [146]:
import time as time
import pandas as pd

for data in [text, decapitalized_text]:
    df = pd.DataFrame(columns = ['usual', 'tricked'], index=[32, 64])
    for compressor in ['usual', 'tricked']:
        for window_size in [32*1024, 64*1024]:
            start_time = time.time()
            size = len(deflate_compressor(data=data, compressor=compressor, window_size=window_size)) / 1024
            required_time = time.time() - start_time

            df.loc[window_size/1024, compressor] = f'{size:.2f} кб, {required_time:.2f} секунд'
    
    print(f'Для {'декапитализированного' if data == decapitalized_text else 'обычного'} текста:')
    print(df, '\n')

Для обычного текста:
                     usual                  tricked
32  400.85 кб, 5.43 секунд   415.72 кб, 7.91 секунд
64  382.10 кб, 7.81 секунд  388.35 кб, 14.16 секунд 

Для декапитализированного текста:
                     usual                  tricked
32  395.53 кб, 5.31 секунд   409.80 кб, 8.10 секунд
64  377.09 кб, 7.90 секунд  382.86 кб, 13.61 секунд 



Выводы:  
1. Увеличение размера окна позволяет улучшить степень сжатия, но при этом увеличивается время сжатия, но не линейно  
  
2. Использование трюка для улучшения качества сжатия не принесло, но уменьшило скорость сжатия из-за отсутствия параллельного вычислений для T[n..] и T[n+1..]    
При этом, если добавить условие, что длина нового повтора должна быть на 5 меньше (а не просто меньше), чем старого, то степень сжатия незначительного улучшается (на 1кб)  
  
3. В ДЗ 2.1 было выяснено, что запись исключений в сжатом виде занимает 2.152 кб.  
При этом разница между объёмом обычного и декапитализированного сжатого текста составляет примерно 5кб в каждом случае,   
то есть декапитализация позволит сэкономить 3кб (0,24% от исходного текста в 1239кб), следовательно формально оправдана

4. Время сжатия декапитализированного и исходного текстов одинаковое

# Для сравнения библиотечная реализация deflate в zlib

In [202]:
import zlib

def lz77_compress(data):
    """Сжатие данных с использованием zlib (LZ77 + Huffman)"""
    return zlib.compress(data, level=zlib.Z_BEST_COMPRESSION)


original_data = text.encode('ascii')
compressed = lz77_compress(original_data)
print(f"Compressed original: {len(compressed) / 1024:.2f} kb")

decapitalized_data = decapitalized_text.encode('ascii')
compressed = lz77_compress(decapitalized_data)
print(f"Compressed decapitalized: {len(compressed) / 1024:.2f} kb")

Compressed original: 374.21 kb
Compressed decapitalized: 368.31 kb
